# Selection algorithms

In [1]:
# Community simulator package
from IPython.display import Image
from community_simulator import *
from community_simulator.usertools import *
from community_simulator.visualization import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends import backend_pdf as bpdf
import numpy as np
import scipy as sp
colors = sns.color_palette()
%matplotlib inline

# Community selection package
from community_selection import *
from community_selection.A_experiment_functions import *
from community_selection.B_community_phenotypes import *
from community_selection.C_selection_algorithms import *
from community_selection.D_migration_algorithms import *
from community_selection.usertools import *

In [2]:
# Make dynanmics by default we will use the microbial consumer resource model
def dNdt(N,R,params):
    return MakeConsumerDynamics(assumptions)(N,R,params)
def dRdt(N,R,params):
    return MakeResourceDynamics(assumptions)(N,R,params)
dynamics = [dNdt,dRdt]

# Global parameters
## Default parameters from community-simulator
## !!!Don't touch this dictionary!!!
assumptions = a_default.copy() # Start with default parameters

## Update parameters for community-selection
assumptions.update({
    'SA': 600*np.ones(3), #Number of species in each specialist family (here, 3 families of 60 species)
    'MA': 30*np.ones(3), #Number of resources in each class
    'Sgen': 300, #Number of generalist species (unbiased sampling over alll resource classes)
    "n_wells": 10,
    "m": 0, # Mortality
    "scale": 10**6,  #scale is a conversion factor specifying the number of individual microbial cells present when N = 1.
    "sigma" : 1, # Standard deviation for drawing specifc speices/interaction function
    "alpha": 1, # Scaling factor between species- and interaction-specific function variances
    "l": 0, # Set leakage function to 0 to switch off cross-feeding
    "response": "type III",
    "sigma_max": 5,
    'R0_food': 1000, # Total amount of supplied food
    "rich_medium": True, # Number of food types passed to R0
    "binary_threshold": 1,  
    # The parameters below will be passed to params_simulation
    "n_propagation": 1, # Length of propagation, or hours within a growth cycle
    "n_transfer": 10, # Number of total transfer, or number of passage
    "n_transfer_selection": 5, # Number of transfer implementing seleciton regimes
    "dilution": 1/1000, # Dilution factor at every transfer
    "n_inoc": 10**6,  #Number of cells sampled from the regional species at start
    "selected_function": "f1_additive"
})

# Prepare experiment setup in this universe
seed_temp = 1
params, params_simulation = prepare_experiment(assumptions, seed = seed_temp)

In [3]:
list_phenotypes = ["f1_additive"]
list_algorithms = ["knock_in_isolates"]
algorithms = make_algorithms(params_simulation)
i = 0; j = 0
params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]]
params_simulation.update({"selected_function": list_phenotypes[j]}) # selected function
assumptions.update({"n_wells": np.sum(assumptions["SA"])  + assumptions["Sgen"]})



# Set seeds
np.random.seed(2)

# Make initial state
init_state = MakeInitialState(assumptions)

# Make plate
plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 

print(params_algorithm)
# Update the community composition by sampling from the pool
print("\nGenerating initial plate")
plate.N = sample_from_pool(plate.N, scale = assumptions["scale"], inocula = params_simulation["n_inoc"], monoculture = True)
if assumptions["rich_medium"]:
    plate.R = make_rich_medium(plate.R, assumptions)
    plate.R0 = make_rich_medium(plate.R, assumptions) # R0 for refreshing media on passaging if "refresh_resoruce" is turned on

plate = add_community_function(plate, dynamics, assumptions, params, params_simulation, params_algorithm)
#community_function = globals()[params_algorithm["community_phenotype"][0]](plate, assumptions = assumptions) # Community phenotype

#community_function

      algorithm_name  transfer community_phenotype selection_algorithm  \
0  knock_in_isolates         1         f1_additive        no_selection   
1  knock_in_isolates         2         f1_additive        no_selection   
2  knock_in_isolates         3         f1_additive        no_selection   
3  knock_in_isolates         4         f1_additive        no_selection   
4  knock_in_isolates         5         f1_additive          select_top   
5  knock_in_isolates         6         f1_additive        no_selection   
6  knock_in_isolates         7         f1_additive        no_selection   
7  knock_in_isolates         8         f1_additive        no_selection   
8  knock_in_isolates         9         f1_additive        no_selection   
9  knock_in_isolates        10         f1_additive        no_selection   

  migration_algorithm  
0        no_migration  
1        no_migration  
2        no_migration  
3        no_migration  
4        no_migration  
5        no_migration  
6        no_migra

Passaging monoculture plate. Transfer 2
            W0   W1   W2   W3   W4   W5   W6   W7   W8   W9  ...  W2090  \
F0  S0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S5     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S6     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S7     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S8     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S9     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S10    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S11    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

Passaging monoculture plate. Transfer 3
            W0   W1   W2   W3   W4   W5   W6   W7   W8   W9  ...  W2090  \
F0  S0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S5     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S6     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S7     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S8     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S9     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S10    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S11    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

Passaging monoculture plate. Transfer 4
            W0   W1   W2   W3   W4   W5   W6   W7   W8   W9  ...  W2090  \
F0  S0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S5     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S6     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S7     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S8     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S9     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S10    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   
    S11    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

Passaging monoculture plate. Transfer 5
W0       0.0
W1       0.0
W2       0.0
W3       0.0
W4       0.0
W5       0.0
W6       0.0
W7       0.0
W8       0.0
W9       0.0
W10      0.0
W11      0.0
W12      0.0
W13      0.0
W14      0.0
W15      0.0
W16      0.0
W17      0.0
W18      0.0
W19      0.0
W20      0.0
W21      0.0
W22      0.0
W23      0.0
W24      0.0
W25      0.0
W26      0.0
W27      0.0
W28      0.0
W29      0.0
        ... 
W2070    0.0
W2071    0.0
W2072    0.0
W2073    0.0
W2074    0.0
W2075    0.0
W2076    0.0
W2077    0.0
W2078    0.0
W2079    0.0
W2080    0.0
W2081    0.0
W2082    0.0
W2083    0.0
W2084    0.0
W2085    0.0
W2086    0.0
W2087    0.0
W2088    0.0
W2089    0.0
W2090    0.0
W2091    0.0
W2092    0.0
W2093    0.0
W2094    0.0
W2095    0.0
W2096    0.0
W2097    0.0
W2098    0.0
W2099    0.0
Length: 2100, dtype: float64
            W0   W1   W2   W3   W4   W5   W6   W7   W8   W9  ...  W2090  \
F0  S0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ..

In [5]:
plate.R0

W0    W1    W2    W3    W4    W5    W6    W7    W8    W9  ...  \
T0 R0   14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  ...   
   R1    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
   R2   10.0  10.0  10.0  10.0  10.0  10.0  10.0  10.0  10.0  10.0  ...   
   R3    8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0  ...   
   R4    8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0  ...   
   R5    8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0  ...   
   R6    3.0   3.0   3.0   3.0   3.0   3.0   3.0   3.0   3.0   3.0  ...   
   R7   13.0  13.0  13.0  13.0  13.0  13.0  13.0  13.0  13.0  13.0  ...   
   R8    8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0   8.0  ...   
   R9    4.0   4.0   4.0   4.0   4.0   4.0   4.0   4.0   4.0   4.0  ...   
   R10  21.0  21.0  21.0  21.0  21.0  21.0  21.0  21.0  21.0  21.0  ...   
   R11  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  ...   
   R12   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
   R13  11.0  11.0  11.0  11.0  11.0  11.0  11.0  11.0  11.0  11.0  ...   
   R14   6.0   6.0   6.0   6.0   6.0   6.0   6.0   6.0   6.0   6.0  ...   
   R15  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  ...   
   R16  24.0  24.0  24.0  24.0  24.0  24.0  24.0  24.0  24.0  24.0  ...   
   R17  18.0  18.0  18.0  18.0  18.0  18.0  18.0  18.0  18.0  18.0  ...   
   R18  25.0  25.0  25.0  25.0  25.0  25.0  25.0  25.0  25.0  25.0  ...   
   R19   2.0   2.0   2.0   2.0   2.0   2.0   2.0   2.0   2.0   2.0  ...   
   R20  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  ...   
   R21   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
   R22  12.0  12.0  12.0  12.0  12.0  12.0  12.0  12.0  12.0  12.0  ...   
   R23   2.0   2.0   2.0   2.0   2.0   2.0   2.0   2.0   2.0   2.0  ...   
   R24   2.0   2.0   2.0   2.0   2.0   2.0   2.0   2.0   2.0   2.0  ...   
   R25  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  14.0  ...   
   R26   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0  ...   
   R27   3.0   3.0   3.0   3.0   3.0   3.0   3.0   3.0   3.0   3.0  ...   
   R28   5.0   5.0   5.0   5.0   5.0   5.0   5.0   5.0   5.0   5.0  ...   
   R29   5.0   5.0   5.0   5.0   5.0   5.0   5.0   5.0   5.0   5.0  ...   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
T2 R60  18.0  18.0  18.0  18.0  18.0  18.0  18.0  18.0  18.0  18.0  ...   
   R61  11.0  11.0  11.0  11.0  11.0  11.0  11.0  11.0  11.0  11.0  ...   
   R62  22.0  22.0  22.0  22.0  22.0  22.0  22.0  22.0  22.0  22.0  ...   
   R63   6.0   6.0   6.0   6.0   6.0   6.0   6.0   6.0   6.0   6.0  ...   
   R64   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0  ...   
   R65   9.0   9.0   9.0   9.0   9.0   9.0   9.0   9.0   9.0   9.0  ...   
   R66   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0  ...   
   R67   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
   R68  13.0  13.0  13.0  13.0  13.0  13.0  13.0  13.0  13.0  13.0  ...   
   R69   9.0   9.0   9.0   9.0   9.0   9.0   9.0   9.0   9.0   9.0  ...   
   R70   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0  ...   
   R71   6.0   6.0   6.0   6.0   6.0   6.0   6.0   6.0   6.0   6.0  ...   
   R72   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0  ...   
   R73   9.0   9.0   9.0   9.0   9.0   9.0   9.0   9.0   9.0   9.0  ...   
   R74  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  ...   
   R75  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  17.0  ...   
   R76   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0   7.0  ...   
   R77   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
   R78  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  ...   
   R79   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
   R80  15.0  15.0  15.0  15.0  15.0  15.0  15.0  15.0  15.0  15.0  ...   
   R81   1.0   1.0   1.0   1.0   1.

In [3]:
data_directory = "data/test/"

#list_phenotypes = ["f6_resident_growth", "f7_resident_growth_community"]
list_phenotypes = ["f8_resident_growth_environment"]
#list_phenotypes = ["f6_resident_growth", "f7_resident_growth_community", "f8_resident_growth_environment"]
#list_phenotypes = ["f1_additive", "f2_interaction", "f3_additive_binary", "f4_interaction_binary", "f5_invader_growth", "f6_resident_growth"]

list_algorithms = ["simple_screening", "resource"]
#list_algorithms = ["simple_screening", "directed_selection_migration", "pair_top_communities", "multiple_pair_top", "Blouin2015", "Mueller2019", "Panke_Buisse2015", "Swenson2000a", "Swenson2000b", "Williams2007a", "Williams2007b", "Wright2019", "coalescence", "migration", "resource", "bottleneck", "knock_out", "knock_in"]


for j in range(len(list_phenotypes)):
    # Parameters for simulation
    params_simulation.update({"selected_function": list_phenotypes[j]}) # selected function

    # Make the list of algorithms
    algorithms = make_algorithms(params_simulation)

    # Simulation
    for i in range(len(list_algorithms)):
        simulate_community(
            assumptions = assumptions,
            params = params,
            dynamics = dynamics,
            params_simulation = params_simulation,
            params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]],
            write_composition = True,
            write_function = True,
            file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i],
            assembly_type = str(list_algorithms[i]),
        )


Algorithm: simple_screening


 transfer             community_phenotype selection_algorithm migration_algorithm
        1  f8_resident_growth_environment        no_selection        no_migration
        2  f8_resident_growth_environment        no_selection        no_migration
        3  f8_resident_growth_environment        no_selection        no_migration
        4  f8_resident_growth_environment        no_selection        no_migration
        5  f8_resident_growth_environment        no_selection        no_migration
        6  f8_resident_growth_environment        no_selection        no_migration
        7  f8_resident_growth_environment        no_selection        no_migration
        8  f8_resident_growth_environment        no_selection        no_migration
        9  f8_resident_growth_environment        no_selection        no_migration
       10  f8_resident_growth_environment        no_selection        no_migration

Generating initial plate

Adding attributes that are essential to 

In [5]:

# Set seeds
np.random.seed(2)

# Make initial state
init_state = MakeInitialState(assumptions)

# Make plate
plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 

# Update the community composition by sampling from the pool
print("\nGenerating initial plate")
#plate.N = sample_from_pool(plate.N, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])

# Update the supplied resource if assumptions["rich_medium"]
#if assumptions["rich_medium"]:
#    plate.R = make_rich_medium(plate.R, assumptions)
#    plate.R0 = make_rich_medium(plate.R, assumptions) # R0 for refreshing media on passaging if "refresh_resoruce" is turned on



Generating initial plate


In [9]:
plate.__dict__
plate.__dict__.keys()

dict_keys(['N', 'R', 'R0', 'S', 'n_wells', 'M', 'dNdt', 'dRdt', 'params', 'dimensions', 'scale', 'parallel', 'species_function', 'interaction_function', 'R0_initial'])

In [6]:
setattr(plate, "n_wells", 20)
#plate.__dict__.keys()
#plate.__dict__
plate.params["S"]

2100

In [14]:
plate.N

W0        W1        W2        W3        W4        W5  \
F0  S0     0.000000  0.000000  0.000000  0.001930  0.000000  0.000000   
    S1     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2     0.000000  0.000000  0.000000  0.003585  0.000000  0.000000   
    S3     0.000000  0.000000  0.000000  0.000000  0.000015  0.000000   
    S4     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S5     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S6     0.000000  0.000000  0.000000  0.005143  0.000000  0.000000   
    S7     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S8     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S9     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S10    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S11    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S12    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S13    0.000000  0.000000  0.000000  0.010712  0.000000  0.000000   
    S14    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S15    0.000000  0.000000  0.000000  0.000000  0.000000  0.018648   
    S16    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S17    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S18    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S19    0.000000  0.000000  0.000000  0.000679  0.000000  0.000000   
    S20    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S21    0.002125  0.000000  0.000000  0.000312  0.000000  0.000000   
    S22    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S23    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S24    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S25    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S26    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S27    0.000000  0.000000  0.000000  0.000000  0.001369  0.000691   
    S28    0.000000  0.000000  0.000000  0.000000  0.000896  0.000000   
    S29    0.000000  0.000000  0.000077  0.000000  0.000000  0.000000   
...             ...       ...       ...       ...       ...       ...   
GEN S2070  0.000000  0.000000  0.000000  0.004832  0.000000  0.000000   
    S2071  0.000000  0.000001  0.026267  0.000000  0.000000  0.000000   
    S2072  0.000000  0.000000  0.000000  0.000136  0.000000  0.000000   
    S2073  0.000000  0.000000  0.000000  0.017114  0.000000  0.000000   
    S2074  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2075  0.000000  0.000000  0.000000  0.001865  0.000001  0.000000   
    S2076  0.000000  0.000000  0.000811  0.000000  0.000000  0.000000   
    S2077  0.000000  0.000000  0.004606  0.000000  0.000000  0.036438   
    S2078  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2079  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2080  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2081  0.000000  0.000000  0.000000  0.000006  0.000000  0.000000   
    S2082  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2083  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2084  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2085  0.000000  0.000000  0.000000  0.000000  0.000000  0.000070   
    S2086  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2087  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2088  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2089  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
    S2090  0.000000  0.000684  0.000000  0.000000  0.000000  0.000000   
    S2091  0.000000  0.000000  0.000000  0.000816  0.000000  0.000000   
    S2092  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
  